In [2]:
import pdfminer.high_level as hl
import re
import os


## Importing Job Description Dataset

In [22]:
import pandas as pd
import os
import random

dataset_path = os.path.join(os.getcwd(),"archive/training_data.csv") #Job_description Dataset file path
df = pd.read_csv(dataset_path)

num_jd = 15 #Number of Job Descriptions we want to take

# Generate random indices to select rows
random_indices = random.sample(range(len(df)), num_jd)

# Select rows from the dataset based on the random indices
jd = [df.iloc[i] for i in random_indices]


# len(jd)

jd=pd.DataFrame(jd)

In [23]:
# jd

,company_name,job_description,position_title,description_length,model_response
750,Thinx Inc. New York City Metropolitan Area ...,art director photo and video\nat thinx view al...,Art Director,6986,"{\n ""Core Responsibilities"": ""Craft conceptu..."
849,Parker,responsibilities\nparkers internship program w...,Human Resources Internship - Corporate (Year-...,3840,"{\n ""Core Responsibilities"": ""Assist in gene..."
647,"HCA Healthcare Jacksonville, FL On-site",\nwhat do you think about this type of role\n\...,Medical Office Specialist,3718,"{\n ""Core Responsibilities"": ""Manage all fro..."
711,Hudson Group,grow with us\nhudson is north americas leading...,Windows System Administrator,3064,"{\n ""Core Responsibilities"": ""Monitoring sys..."
59,Accenture,who we are\n\naccenture is a global profession...,Workday Sales Director,6936,"{\n ""Core Responsibilities"": ""Manage the opp..."
389,Certified Oil,are you a high energy outgoing person who love...,Retail Sales Associate,3160,"{\n ""Core Responsibilities"": ""Greeting guest..."
440,Marcus & Millichap,mission capital advisors mca a subsidiary of m...,Network Systems Admin,4679,"{\n ""Core Responsibilities"": ""Administer day..."
115,Columbus Business First,total compensation includes base uncapped com...,Multi-Media Advertising Sales Executive,3656,"{\n ""Core Responsibilities"": ""Work with mark..."
755,Insider Inc,insider inc is looking for an experienced crea...,"VP, Marketing - Advertising Revenue",5232,"{\n ""Core Responsibilities"": ""Lead marketing..."
521,Cynet Systems,we are looking for senior embedded c developer...,Senior Embedded C++ Developer,2328,"{\n ""Core Responsibilities"": ""Responsible fo..."


### This code block extracts Job Description and extracts details like required skills and educational qualifications required for the job

In [12]:
import json

# Function to clean and convert a single 'model_response' string into a dictionary
def JD_Extractor(response):
    # Clean the string to remove Python-style comments (e.g., 'N/A')
    cleaned_response = re.sub(r'"\s*N/A\s*"', '""', response)
    # Parse the JSON-like string into a dictionary
    response_dict = json.loads(cleaned_response)
    return response_dict



# Apply the process_response function to each row in 'model_response' column
jd['parsed_response'] = jd['model_response'].apply(JD_Extractor)

# Create a new DataFrame with the parsed results
response_df = pd.json_normalize(jd['parsed_response'])

# Rename columns if needed
response_df = response_df.rename(columns={
    'Required Skills': 'required_skills',
    'Educational Requirements': 'educational_requirements',
})

jd_df = response_df[['required_skills','educational_requirements','Preferred Qualifications']]
jd_df
        
        


# This is the main PDF Extractor function which extracts the key details:
# Category (Job role)
# Skills
# Education (Degree, Institution)


In [24]:
def PDFExtractor(resume):
    try:
        # Regular expressions for skills extraction
        # Initializing regex patterns to search
        skills_pattern =   r"Skills([\s\S]*?)(?=\\n\\n[Work])|Skills([\s\S]+)|Skills([\s\S]+?)(?=\\n\\n[Experience])|Skills([\s\S]+?)(?=\\n\\n[Education])"
        education_pattern = r"Education([\s\S]+?)(?=\n[A-Z])"
        category_pattern = r"(\b[A-Z\s]+\b)(?=\n)"

        skills_matches = re.finditer(skills_pattern, resume, re.MULTILINE | re.UNICODE | re.DOTALL)
        education_matches = re.finditer(education_pattern, resume, re.MULTILINE | re.UNICODE)
        category_matches = re.search(category_pattern, resume, re.MULTILINE | re.UNICODE)

         # Initialize variables
        skills = None
        education = None
        
################################################################################################################################        
        # Job Role Extraction:
        if category_matches:

            for groupNum in range(0, len(category_matches.groups())):
                groupNum = groupNum + 1

                print (f"Job Role: {category_matches.group(groupNum)}")


        category = category_matches.group(groupNum)      

################################################################################################################################        
       # Latest Education Details Extraction
        for matchNum, match in enumerate(education_matches, start=1):
            
            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1

                print (f"Education: {match.group(groupNum)}")
        if match.group(groupNum) is not None:
            education = match.group(groupNum)
        else:
            education = None

################################################################################################################################        
            
        # Skills Extraction
        for matchNum, match in enumerate(skills_matches, start=1):

        #     print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1
                if match.group(groupNum) is not None:
                    print(f"Skills: {match.group(groupNum)}")
                    skills = match.group(groupNum)
#                     skills = skills.replace('\n','').replace(' ','').split(',')
                
    except Exception as e:
        print(e)
        pass

################################################################################################################################        
        
    # Create a DataFrame from the extracted data
    resume_data = pd.DataFrame({
        'Job Role': [category],
        'Education': [education],
        'Skills': [skills]
    })

    return resume_data

### This code block takes out the file_paths of all the resumes with us
### Then storing the categories of jobs we have with us
### Then storing pdf file names we have with us 

In [ ]:
root_directory = os.path.join(os.getcwd(),'archive/data/data')
file_paths = []
for root,_,files in os.walk(root_directory):
    for file in files:
        file_paths.append(os.path.join(root, file))
print(file_paths)


import random
file_paths_shuffled = random.shuffle(file_paths)



cats = []
for path in file_paths_shuffled:
    cats.append(os.path.basename(os.path.dirname(path)))

#S
pdfs = []
for file in file_paths_shuffled:
    pdfs.append(file.split('\\')[-1])

################################################################################################################################

# This code block has 2 parts 
### Firstly, it calls our PDFExtractor function, extracts all the information mentioned above from all the resumes and creates a resume dataframe
### Secondly, it adds Category (Job category) and Pdf filenames in to the dataframe corresponding to each individual resume detail


In [17]:
def process_resume(file_paths_shuffled):
    combined_resume_data = pd.DataFrame(columns=['Job Role','Education','Skills'])
    for file_path in file_paths_shuffled:  
        with open(file_path, "rb") as f:
            resume = hl.extract_text(f)
        resume_data = PDFExtractor(resume)
    #     skill[os.path.basename(os.path.dirname(file_path))] = resume_data['Skill']
        combined_resume_data = pd.concat([combined_resume_data, pd.DataFrame(resume_data)], ignore_index=True)
        print("resume_data:", resume_data)
    return combined_resume_data


    
combined_resume_data = process_resume(file_paths_shuffled)  
combined_resume_data['Category'] = cats
combined_resume_data['Files'] = pdfs

Job Role: SALES REPRESENTATIVE
Education:  and Training
Skills: 

Sales Software: Salesforce.com, TapScan
Public Relations Software: Bacon's Mediasource, Factiva
Microsoft Office Suite: Outlook, MS Word, Excel, Powerpoint
Human Resources: EZ Labor Time and Attendance Software, ADP Payroll System, Great Plains Software


resume_data:                Job Role      Education  \
0  SALES REPRESENTATIVE   and Training   

                                              Skills  
0  \n\nSales Software: Salesforce.com, TapScan\nP...  
Job Role: EXECUTIVE CHEF
Education:  and Abilities as a Chef where my vast experience and demanding work ethic will be
effectively utilized. Strong and diverse experience in the HOSPITALITY INDUSTRY, including: l

Education: 
Bachelor of Science degree : Culinary Management , 2006 ART INSTITUTE OF CALIFORNIA ï¼​ City , State Culinary Management
Skills: 
ABC, Photoshop, basic, Computer-literate, Ventura, costing, customer relations, French, General Manager, hiring, 

Job Role: SENIOR ACCOUNTANT
Education: 
Master of Accounting & Financial Management : 2012 Keller Graduate School of Management of DeVry University - City , State GPA: GPA:
3.91 GPA: 3.91
Skills: 

Strong communication skills
ERP (Enterprise Resource Planning) software
Certified Public Accountant (CPA)
QuickBooks expert
Superior attention to detail
Account reconciliation specialist
MS Office Suite

Budget analysis
Self-motivated professional
Advanced bookkeeping skills
SAP expertise
MS Office Suite

Work History
Senior Accountant 06/2011 to Current 
Company Name â€“ City , State 

Management of high level accounts including 280 endowment records totaling $85+ million and a private golf club account.
Oversight includes general ledger account activity, gift agreements, schedules, reconciliation, wire transfers, journal entries, and the ongoing
accurate analysis of all financial activity.
Record monthly investment activity, including purchases, sales, interest, dividends, fees, as well as

Job Role: CONSTRUCTION MANAGER
Education: 
2002
Skills: 

acquisitions, budget preparation, budgets, budget, cables, Cable, coaching, continuous improvement, CPR, customer service, documentation,
drafting, engineer, filing, First Aid, forms, frame, inventory, Inventory Control, invoicing, marketing, materials, Meetings, office, win, monitors,
Operations management, personnel, presentations, pricing, processes, Program Development, progress, Project Management, proposals, quality,
Radio, Safety, Supervision, system design, technician, Telecommunications, telephony, Transmission, upgrade


resume_data:                Job Role Education  \
0  CONSTRUCTION MANAGER    \n2002   

                                              Skills  
0  \n\nacquisitions, budget preparation, budgets,...  
Job Role: HEALTHCARE CONSULTANT
Education: 
Master of Business Administration Bachelor of Science : Business Administration/Management Health Services Administration , 2000 Kennesaw
Skills: 

acquisitions, 

Job Role: APPAREL ZONE SUPERVISOR
cannot access local variable 'match' where it is not associated with a value
resume_data:                   Job Role Education Skills
0  APPAREL ZONE SUPERVISOR      None   None
Job Role: STORE MANAGER
Education: al Background
Skills: 
billing, oral communication, negotiate contracts, clients, customer service, English, general manager, hiring, HR, Inventory, Korean, team leader,
director, Microsoft Excel, MS Word, people skills, problem solving skills, speaking, Real Estate, rehabilitation, reporting, sales, store manager,
phones, trainer, trouble shooting, typing skills, Excellent written


resume_data:         Job Role      Education  \
0  STORE MANAGER  al Background   

                                              Skills  
0  \nbilling, oral communication, negotiate contr...  
Job Role: HR COORDINATOR
Education: 
2008 DePaul University - City , State BA Human Resources
Skills:  and Qualifications

Microsoft Sharepoint, SAP, Sharepoint, Excel, Gr

Job Role: CULINARY ARTS INSTRUCTOR
Education:  or Management. So that I can utilize my skills gained during the
past 16 years as a Teacher, Supervisor, Trainer, Team Player and Problem-Solver.
Education: 
Master of Science : Family and Consumer Science , 2012 Queens College CUNY ï¼​ City , State
Skills: 
administrative, advertising, arts, budgeting, coaching, customer service, training employees, special events, financial, food safety, hiring, instruction,
Inventory, inventory control, leadership skills, lesson plans, marketing, materials, Microsoft Office, presentations, progress, purchasing, quality,
Quality control, receiving, safety, sales, SAP, shipping, supervisor, teacher, teaching


resume_data:                    Job Role  \
0  CULINARY ARTS INSTRUCTOR   

                                           Education  \
0  \nMaster of Science : Family and Consumer Scie...   

                                              Skills  
0  \nadministrative, advertising, arts, budgeting...  


Job Role: CHEMICAL ENGINEERING INTERN
Education: 

2018
Skills:  and Qualifications

Monitor and analyze data from processes and experiments.

Follow safety procedures to be employed by workers operating equipment or working in close proximity to ongoing chemical reactions.

Improve processes to separate components of liquids or gases or generate electrical currents, using controlled chemical processes.

Troubleshoot problems with chemical manufacturing processes.

Evaluate chemical equipment and processes to identify ways to optimize performance or to ensure compliance with safety and environmental
regulations.

Research engineering aspects of biological or chemical processes.

Conduct validation tests of equipment or processes.

Determine causes of operational problems or failures.

Evaluate characteristics of equipment or systems.

Proficient in Microsoft Office, OAM, MES, and EPI.

Working knowledge of SAP, ChemCad, C+, Java.

Advanced written and conversational German.

Intermedi

Job Role: NETWORK SERVICES INTERN
Education: 
Bachelor of Science Information Science & Technology University of Wisconsin City , State | 12-2014 Information Science & Technology
Skills: 
Â·Able to research and familiarize skill sets towards core technologies
based on enthusiasm, dedication, and motivation
Â·Knowledgeable with installation and configuration of Windows-
based operating systems (Windows XP, Vista, 7, and 8)
Â·Experienced with the installation and configuration of Ubuntu Linux
whilst cognizant of Mac OS X interactions
Â·Proficient with virtualization and highly skilled with configurations
inside of VMware
Work History
Company Name Support & Network Services Intern | City , State | September 2014 - December 2014

Â·Skilled with troubleshooting hardware and software issues within a
network and support services environment
Â·Able to document, effectively communicate, and succeed within a
group setting in order to become a superior team player 
Willing to work individually wh

Job Role: GRAPHIC DESIGNER
Education: 
High School Diploma : Graphics/IT , 2017 Forest Park High School ï¼​ City , State , United States

Skills: 

Meets deadlines
Excellent Multi-tasking
Planning and Arranging activities
Conversant and fluent in English, Spanish, and Vietnamese

Desktop Publishing Software: Photoshop, Illustrator
Active Listening Skills
Advanced Problem-Solving
Great Organizational Skills
Confidence and Empathy

Interests

Art
Photography
Graphic Design
Writing
Reading
Volunteering

Languages

I am able to comfortably speak and write in English. I am also able to speak Spanish, but not as skilled in writing. I am currently in my 3rd year in
Spanish Classes, and I was given the opportunity to join the Spanish Honor Society at my school recently. And I can also speak Vietnamese on a
fluent level, but I am not able to write in Vietnamese.

Accomplishments

Being offered the opportunity to create a graphic design piece for a school event, and having my artwork chosen for 

Job Role: MULTIMEDIA SPECIALIST
Education: 
Bachelor of Arts : Journalism , September 1994 The Ohio State University School of Journalism - City , State

Skills: 

Social media, digital video and multimedia, content curation, digital strategy, web metrics and SEO optimization, digital engagement, community
engagement, public relations,


resume_data:                 Job Role                                          Education  \
0  MULTIMEDIA SPECIALIST  \nBachelor of Arts : Journalism , September 19...   

                                              Skills  
0  \n\nSocial media, digital video and multimedia...  
'NoneType' object has no attribute 'group'


UnboundLocalError: cannot access local variable 'category' where it is not associated with a value

## The following function cleans both of our datasets. (Job Descriptions and CVs/Resumes dataset)

In [ ]:
import contractions
import nltk
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Convert the frozen set to a list
sklearn_stopwords = set(ENGLISH_STOP_WORDS)

# Combining the stopwords form sklearn & NLTK
combined_stopwords = sklearn_stopwords.union(nltk_stopwords)

#Cleaning Jd and Resume
def cleaning_text(text):
    new_sent = ''
    try:
        cleaned_text = contractions.fix(text)

        # User-mentions Removal
        cleaned_text = re.sub("@[A-Za-z0-9]+", "", cleaned_text)

        # Hashtag Removal
        cleaned_text = re.sub("#", "", cleaned_text)

        #Hyperlink Removal
        cleaned_text = re.sub(r"http\S+","", cleaned_text)

        # Punctuation, Special Characters and digits Removal (Retaining only the alphabets)
        #     cleaned_text = re.sub(r"[^a-zA-Z]", " " , cleaned_text )

        # convert the tweet into lowercase & get rid of any leading or trailing spaces
        cleaned_text = cleaned_text.lower().strip()

        #remove stopwords from the new_sent
        cleaned_tokens = [token for token in cleaned_text.split() if token not in combined_stopwords]

        # Retain only those token which have length > 2 characters
        cleaned_tokens = [token for token in cleaned_tokens if len(token)>2]

        for token in cleaned_tokens:
            new_sent = new_sent + token + ' '
    
    except:
        new_sent = None
        pass
            
            
    return new_sent


cleaned_resume_data=combined_resume_data[['Skills','Education','Category']].map(cleaning_text)
cleaned_jd=jd[["job_description","model_response"]].map(cleaning_text)

# This is our main CV-JD matching function

## It firstly tokenizes our Job description and CV details
### "required skills" and "educational requirements" for Job description and
### "Skills" and "Education" of the candidate (extracted from resume)

## It then embeds all the four details using DistilBERT Transformer Model.

## It then matches the Candidate skills, taken from candidate resume, with skills required for the job from JD and latest educational qualification of the candidate with educational requirements for the job from the JD, respectively.

## I have made use of Cosine similarity function to measure the proximity of the Candidate skills to Skills required for job and Candidate's educational qualifications to educational requirements for the job 

### I have then multiplied the 2 cosine similarity matrices to get to the final cosine similarity matrix to measure the match rating of the candidate's resume to the job description
### The reasoning behind this is to segregate highly good resumes, in both the metrics (skill + education), from  poor resumes as well as to incentivize resumes with a good score in one of the criteria over poor resumes but also punish them as compared to resumes with good score in both the metrics 

## The function then ranks the resumes for each job description accordingly and displays them

In [ ]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig
from sklearn.metrics.pairwise import cosine_similarity


def CV_JD_matching(resume_data,jd):
    # Initialize the DistilBERT tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    model = DistilBertModel.from_pretrained("distilbert-base-uncased")
    jd_df_skills = jd['required_skills'].tolist()
    jd_df_edu = jd['educational_requirements'].tolist()

    crd_skills = resume_data['Skills'].tolist()
    crd_edu = resume_data['Education'].tolist()
    # Tokenize and preprocess job descriptions
    job_desc_tokenized_skill = tokenizer(jd_df_skills, truncation=True, padding=True, return_tensors="pt")
    job_desc_tokenized_edu = tokenizer(jd_df_edu, truncation=True, padding=True, return_tensors="pt")

    # Tokenize and preprocess CVs
    cvs_tokenized_skill = tokenizer(crd_skills, truncation=True, padding=True, return_tensors="pt")
    cvs_tokenized_edu = tokenizer(crd_edu, truncation=True, padding=True, return_tensors="pt")

    
    
    # Get embeddings for job descriptions and CVs for skill metric
    job_desc_embeddings_skill = model(**job_desc_tokenized_skill).last_hidden_state[:, 0]
    job_desc_embeddings_skill = job_desc_embeddings_skill.detach().cpu().numpy()  # Convert to NumPy array
    print(f"{type(job_desc_embeddings_skill)}")
    # Stack CV embeddings into one array
    cv_embeddings_skill = model(**cvs_tokenized_skill).last_hidden_state[:, 0]
    cv_embeddings_skill = cv_embeddings_skill.detach().cpu().numpy()  # Convert to NumPy array
    print(f"{type(cv_embeddings_skill)}")
    # Calculate cosine similarity for each job description and CV pair
    similarity_scores_skill = cosine_similarity(job_desc_embeddings_skill, cv_embeddings_skill)
    
     # Get embeddings for job descriptions and CVs for skill metric
    job_desc_embeddings_edu = model(**job_desc_tokenized_edu).last_hidden_state[:, 0]
    job_desc_embeddings_edu = job_desc_embeddings_edu.detach().cpu().numpy()  # Convert to NumPy array
    print(f"{type(job_desc_embeddings_edu)}")
    # Stack CV embeddings into one array
    cv_embeddings_edu = model(**cvs_tokenized_edu).last_hidden_state[:, 0]
    cv_embeddings_edu = cv_embeddings_edu.detach().cpu().numpy()  # Convert to NumPy array
    print(f"{type(cv_embeddings_edu)}")
    # Calculate cosine similarity for each job description and CV pair
    similarity_scores_edu = cosine_similarity(job_desc_embeddings_edu, cv_embeddings_edu)
    
#     print(f"similarity_scores_edu: {similarity_scores_edu}")
#     print(f"similarity_scores_skill: {similarity_scores_skill}")
    
    similarity_scores_total = similarity_scores_skill*similarity_scores_edu
    
    # Rank CVs based on similarity scores for each job description
    ranked_cvs = []
    for i, job_desc in enumerate(jd):
        job_desc_similarity_scores = similarity_scores_total[i]
        ranked_cv_indices = np.argsort(job_desc_similarity_scores)[::-1][:5]  # Get top 5 CV indices
        ranked_cvs.append([resume_data.iloc[j][['Skills','Education','Category']] for j in ranked_cv_indices])

    # List the top 5 CVs for each job description
    for i, job_desc in enumerate(jd):
        print(f"Top 5 CVs for Job Description-: {jd.index[i]}:\n")
        for j, cv_text in enumerate(ranked_cvs[i]):
            print(f"Rank {j + 1}: {cv_text}\n")
            print(f"Category: {ranked_cvs[i][2]}:\n")
#             print(f"Cosine Similarity score: {job_desc_similarity_scores}")
                  
                  
 
    
        

    
    

resume_batches = []
cleaned_resume_data=cleaned_resume_data.dropna()
jd_df=jd_df.dropna()
for i in range(0, len(cleaned_resume_data), 15):
    resume_batches.append(cleaned_resume_data[i:i + 15])

for res in resume_batches:
    cleaned_resume_data=res.dropna()
    jd_df=jd_df.dropna()
    # jd_df.shape
    CV_JD_matching(cleaned_resume_data,jd_df)

## Challenges and Suggestions

##### There were plenty of challenges and hurdles faced. 
##### Firstly, one has to observe and analyze the pdf files to understand how to extract Job Roles, Education and Skill set of a particular candidate. Extraction of Skills is still a major problem and manually annotating, embedding and then NER will give a better accuracy in extracting skill details. 
##### Similarly, Job description dataset has a column named "model_response" that contained data of detailed data like required skills and educational requirements in the form of a dictionary. I had to parse and extract it and that gave me a much better matching result.
##### DistilBERT is a good transformer model for the embedding task. But it only takes a maximum of 512 tokens at a time and hence isn't useful for long textual data. Longformer transformer model can overcome that issue.

##### There were a few other tasks that could have been done and challenges that were facced by me. However, these are the major challenges I came up with.